In [1]:
from dotenv import load_dotenv
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import base64
import re
from IPython.display import display, Markdown,Image
import json
from pathlib import Path
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

# 加载 .env 文件
load_dotenv()

# 获取环境变量
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_endpoint = os.getenv("LANGCHAIN_ENDPOINT")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")

api_key=os.getenv("API_KEY")
base_url=os.getenv("BASE_URL")
# deepseek_api_key=os.getenv("DEEPSEEK_API_KEY")
# deepseek_base_url=os.getenv("DEEPSEEK_BASE_URL")

In [2]:
def inputPrompt (question):
       
    # 构建系统消息
    system_message = SystemMessage(
        content="You are a financial expert. You will be given questions and options, possibly with context information and images. Please answer the question."
    )

    # 构建用户消息
    human_message=HumanMessage(content=[])

    if len(question["Share Context"]) != 0:
        human_message.content.append({"type": "text", "text": "Context: " + question["Share Context"]})

    if len(question["Share Image"])!= 0:
        for path in question["Share Image"]:
            image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+path
            with open(image_url, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode("utf-8")
            human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Question: "+ question["Question Text"]})

    if len(question["Image"]) != 0:
        image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+question["Image"]
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
        human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Options: " + str(question["Options"])})

    human_message.content.append({"type": "text", "text": "Let's think step by step. The output reasoning steps are in Markdown format. Finally, must put the correct option (A, B, C, or D) in【 】. e.g.Therefore, the correct option is 【B】."})

    response = [system_message, human_message]
    return response



##Utils
def FeedbackPrompt (question):
       
    system_message = SystemMessage(
            content="""You are a financial expert. You will be given questions and options, possibly with context information and images. Also, you will be given wrong reasoning steps and correct reasoning hints.You are supposed to give feedback.""")

    # 构建用户消息
    human_message=HumanMessage(content=[])

    if len(question["Share Context"]) != 0:
        human_message.content.append({"type": "text", "text": "Context: " + question["Share Context"]})

    if len(question["Share Image"])!= 0:
        for path in question["Share Image"]:
            image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+path
            with open(image_url, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode("utf-8")
            human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Question: "+ question["Question Text"]})

    if len(question["Image"]) != 0:
        image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+question["Image"]
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
        human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Options: " + str(question["Options"])})
    human_message.content.append({"type": "text", "text": "Wrong Reasoning Steps: " + question["Model Reasoning"]})
    human_message.content.append({"type": "text", "text": "Wrong Answer: " + question["Model Answer"]})
    human_message.content.append({"type": "text", "text": "Correct Reasoning Steps: " + question["Explanation"]})
    human_message.content.append({"type": "text", "text": "Correct Answer: " + question["Answer"]})

    human_message.content.append({"type": "text", "text": """ Please give the feedback in Markdown format. 1. Please output correct reasoning steps according to hints. 2. compare the correct reasoning step with the model's wrong reasoning step, and point out the difference. 3. summarize the hint for future simalar questions."""})

    response = [system_message, human_message]
    return response


def ICLPrompt (question,example):
       
    # 构建系统消息
    system_message = SystemMessage(
        content="You are a financial expert. You will be given previous learning document including questions and options, possibly with context information and images. Please answer the current question."
    )

    # 构建用户消息
    human_message=HumanMessage(content=[])
    human_message.content.append({"type": "text", "text": "Previous Learning Document: "})
    if len(example["Share Context"]) != 0:
        human_message.content.append({"type": "text", "text": "Context: " + example["Share Context"]})

    if len(example["Share Image"])!= 0:
        for path in example["Share Image"]:
            image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+path
            with open(image_url, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode("utf-8")
            human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Question: "+ example["Question Text"]})

    if len(example["Image"]) != 0:
        image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+example["Image"]
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
        human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Options: " + str(example["Options"])})
    human_message.content.append({"type": "text", "text": "Wrong Reasoning Steps: " + example["Model Reasoning"]})
    human_message.content.append({"type": "text", "text": "Feedback: " + example["Feedback"]})


    human_message.content.append({"type": "text", "text": "Current Question is as follows: "})
    if len(question["Share Context"]) != 0:
        human_message.content.append({"type": "text", "text": "Context: " + question["Share Context"]})

    if len(question["Share Image"])!= 0:
        for path in question["Share Image"]:
            image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+path
            with open(image_url, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode("utf-8")
            human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Question: "+ question["Question Text"]})

    if len(question["Image"]) != 0:
        image_url = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+question["Image"]
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
        human_message.content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}})

    human_message.content.append({"type": "text", "text": "Options: " + str(question["Options"])})

    human_message.content.append({"type": "text", "text": "Let's think step by step. The output reasoning steps are in Markdown format. Finally, must put the correct option (A, B, C, or D) in【 】. e.g.Therefore, the correct option is 【B】."})

    response = [system_message, human_message]
    return response


In [3]:
from typing import List
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import Generation
from IPython.display import display, Markdown

class MarkdownParser(BaseGenerationOutputParser[str]):
    """
    A custom parser that formats the model output for Markdown display
    by replacing LaTeX-style delimiters \[ and \] with $.
    """
    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """Parse the model output and format it as Markdown.

        Args:
            result: A list of Generations (assumed to contain only one string).
            partial: Whether to allow partial results (for streaming, not used here).

        Returns:
            A Markdown-formatted string with LaTeX-style delimiters replaced.
        """
        # Ensure there's only one generation
        if len(result) != 1:
            raise ValueError("This parser only supports a single generation.")
        
        # Extract the generation content
        generation = result[0]
        if not isinstance(generation.text, str):
            raise ValueError("Expected text output for Markdown formatting.")
        
        # Replace  \\[ and \\] with $ for LaTeX-style display
        formatted_text = generation.text.replace('\\[', '$').replace('\\]', '$').replace('\\(', '$').replace('\\)', '$')
        
        import re



        return formatted_text
    

import re
from langchain.tools import Tool

def extract_answer(text: str) -> str:
    """Extract the answer option (A, B, C, or D) in brackets from the given text."""
    # Regular expression to find the answer in brackets, e.g., [C]
    match = re.search(r"\【([A-D])\】", text)
    if match:
        return match.group(1)  # Returns the answer option (e.g., "C")
    else:
        return "Answer not found"  # Returns a message if no answer is found

# Wrap extract_answer in a LangChain Tool to make it invokable
extract_answer_tool = Tool.from_function(
    func=extract_answer,
    name="Extract Answer Tool",
    description="Extracts the answer option in brackets (e.g., 【C】) from the provided text."
)

import json
import os

def write_output(data, file_path):
    # 如果文件存在，先读取现有数据
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    else:
        existing_data = []

    # 合并新数据到现有数据中
    if isinstance(existing_data, list):
        existing_data.append(data)
    else:
        existing_data = data

    # 将合并后的数据写入到 JSON 文件
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)


In [5]:
# 初始化模型
LLMmodel = ChatOpenAI(model="gemini-1.5-pro-latest", api_key=os.getenv("API_KEY"), base_url=os.getenv("BASE_URL"))
outputParser=MarkdownParser()
chain = LLMmodel|outputParser

In [6]:
#配置文件路径
ErrorLogPath="/Volumes/Jennie/Reasoning/FinReasoning/errorLog/gemini_1.5_pro_ErrorLog.json"
ModelOutputPath="/Volumes/Jennie/Reasoning/FinReasoning/output/gemini_1.5_pro_output.json"
ModelRAGOutputPath="/Volumes/Jennie/Reasoning/FinReasoning/output/gemini_1.5_pro_rag_output.json"

In [8]:
# 加载数据

file_path = '/Volumes/Jennie/Reasoning/FinReasoning/dataset/data_v3.json'
data = json.loads(Path(file_path).read_text())

filtered_data = [item for item in data if item.get("Train") == 1.0]# 设置只跑部分数据
data=filtered_data


Retrieval

In [9]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown,Latex, display
from sentence_transformers import SentenceTransformer
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import faiss
import torch
import pandas as pd
import json
import faiss
import numpy as np

# 加载CLIP模型和处理器
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
#初始化数据库
def init_faiss():
    index = faiss.IndexFlatIP(1024)  # 使用内积度量
    if index.ntotal==0:
        # 初始化 Faiss 索引，使用内积 (dot product) 作为距离度量
        pass
    # 清空Faiss索引中的所有向量
    else:
        index.reset()
    # 检查索引是否清空
    print("Number of vectors after reset:", index.ntotal)
    return index
def index_faiss(index, file_path):
    # 检查文件是否存在
    if os.path.exists(file_path):
        # 文件存在，加载数据
        data = json.loads(Path(file_path).read_text())
    else:
        # 文件不存在，创建一个新的空列表
        data =     [{"ID": 9999,"Question Number": 9999,"Share Context": "","Share Image": "","Question Text": "text","Image": "images/QuantitativeAnalysis1_images/40u.png",
                     "Options": {"A": " -0.215","B": " -0.113","C": " 0.113","D": " 0.215"},"Answer": "C","Explanation": "text","QA Type": "text","Question Type": "text","Level of Difficulty": "text",
                    "Knowledge Topics": "text","General Topics":"text","Book Label": "text","Model Answer": "C","Model Reasoning": "text","Feedback": "text"
                    }]
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
    # 处理数据
    for error in data:
        storeEmbedding(index, error)

    print("Number of vectors after adding:", index.ntotal)
    return index
def clipEmbedding(data):
    textdata = "Question:" + data.get("Question Text") + " Options:" + str(data.get("Options")) + " Correct Answer:" + data.get("Answer")
    
    # 检查是否有图片
    if data.get("Image") != '':
        image_path = "/Volumes/Jennie/Reasoning/FinReasoning/dataset/"+ data.get("Image")
        # print(image_path)
        image = Image.open(image_path)
        # print(image)
        
        # 生成文本和图像的嵌入，添加 truncation=True 和 max_length=77
        inputs = processor(text=[textdata], images=image, return_tensors="pt", padding=True, truncation=True, max_length=77)
        
        # 使用CLIP模型生成嵌入
        outputs = model(**inputs)
        image_embedding = outputs.image_embeds  # 图像嵌入
        text_embedding = outputs.text_embeds  # 文本嵌入
    else:
        # 如果没有图像，生成文本嵌入
        inputs = processor(text=[textdata], return_tensors="pt", padding=True, truncation=True, max_length=77)
        text_embedding = model.get_text_features(**inputs)
        
        # 创建一个与图像嵌入维度相同的零向量
        image_embedding = torch.zeros((text_embedding.shape[0], 512))  # 假设图像嵌入维度是512
    
    # 将文本和图像嵌入拼接在一起
    combined_embedding = torch.cat((text_embedding, image_embedding), dim=-1)
    
    return combined_embedding


def normalize(embeddings):
    # 归一化函数，计算余弦相似度时将向量进行归一化
    norms = torch.norm(embeddings, dim=1, keepdim=True)  # 计算每个向量的范数
    return embeddings / norms  # 将向量归一化，使其范数变为1


# 生成嵌入
def storeEmbedding(index,data):
    error_log_embedding = clipEmbedding(data)
    # 对嵌入进行归一化，以便计算余弦相似度
    error_log_embedding = normalize(error_log_embedding)
    # 将生成的多模态嵌入转换为numpy数组并添加到Faiss索引中
    error_log_embedding_np = error_log_embedding.detach().numpy()  # 确保转换为numpy格式
    index.add(error_log_embedding_np)  # 将嵌入添加到Faiss索引中
    return index

# 查询函数
def query_embedding_faiss(query_data, index, k=5):
    # 生成查询嵌入
    query_embedding = clipEmbedding(query_data)
    query_embedding = normalize(query_embedding)  # 归一化查询向量
    
    # 转换为 numpy 格式
    query_embedding_np = query_embedding.detach().numpy()
    
    # 检索 Faiss 中与查询向量最相似的 k 个向量
    D, I = index.search(query_embedding_np, k)  # D 是余弦相似度，I 是对应的索引

    
    return D, I

In [10]:
index=init_faiss()# 初始化数据库
index_faiss(index,ErrorLogPath) #把现在的errorlog加入到向量数据库

for question in data:
    reasoning=chain.invoke(inputPrompt(question))
    answer=extract_answer(reasoning)    

    modelOutput=question
    modelOutput["Model Answer"]=answer
    modelOutput["Model Reasoning"]=reasoning
    write_output(modelOutput, ModelOutputPath)

    if answer == question["Answer"]:
        write_output(modelOutput, ModelRAGOutputPath)
    else:
        cos,I = query_embedding_faiss(question, index, k=5)
        erroRLog=json.loads(Path(ErrorLogPath).read_text())
        errorexample=erroRLog[I[0][0]]
        reasoning=chain.invoke(ICLPrompt(question,errorexample))
        answer=extract_answer(reasoning)
        display(Markdown(reasoning))
        print("=====================================")
        modelOutput["Model Answer"]=answer
        modelOutput["Model Reasoning"]=reasoning  
        write_output(modelOutput, ModelRAGOutputPath) 

        feedback=chain.invoke(FeedbackPrompt(modelOutput))
        modelOutput["Feedback"]=feedback
        write_output(modelOutput, ErrorLogPath)

Number of vectors after reset: 0
Number of vectors after adding: 20


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


To determine the cumulative translation loss based on Exhibit 1 and Note 3 in Exhibit 2, let's follow these steps:

### Step 1: Identify the Initial Investment
- The parent company transferred **NVK15 million** to Cendaró on 1 January 2016 to purchase a patent for **CRG85.17 million**.

### Step 2: Determine Exchange Rates
- **Exchange Rate on 1 January 2016** (approximation since exact rate isn't given): Use the spot rate closest to this date, which is **5.6780 CRG/NVK** on 31 December 2015.
- **Exchange Rate on 31 December 2016**: **8.6702 CRG/NVK**.

### Step 3: Calculate the Cumulative Translation Adjustment
1. **Calculate the initial NVK value using the exchange rate on 1 January 2016:**
   $
   \text{NVK value} = \frac{\text{CRG85.17 million}}{5.6780} = \text{NVK15 million (as given)}
   $

2. **Calculate the NVK value at year-end using the exchange rate on 31 December 2016:**
   $
   \text{Year-end NVK value} = \frac{\text{CRG85.17 million}}{8.6702} = \text{NVK9.81 million}
   $

3. **Determine the translation loss:**
   $
   \text{Translation loss} = \text{Initial NVK value} - \text{Year-end NVK value} = \text{NVK15 million} - \text{NVK9.81 million} = \text{NVK5.19 million}
   $

### Conclusion

The cumulative translation loss should be the difference calculated above. Since this doesn't match any of the options directly, the discrepancy might be due to the use of a different method or rate for exact computations. However, if options A, B, and C are considered, the closest logical choice based on a rounded or adjusted calculation method could be:

Therefore, the correct option is 【C】.

To determine the total intangible assets in Bindiar francs (FB), we need to calculate the conversion of NVK3 million into FB using the appropriate exchange rate.

### Steps:

1. **Identify the Intangible Assets in NVK:**
   - Total intangible assets are NVK3 million as of 15 July 2016.

2. **Find the Appropriate Exchange Rate:**
   - Use the spot exchange rate on 15 July 2016: 4.1790 FB/NVK.

3. **Convert the Intangible Assets to FB:**
   $
   \text{Intangible Assets in FB} = \text{NVK3 million} \times 4.1790 \text{ FB/NVK}
   $
   $
   \text{Intangible Assets in FB} = 12.537 \text{ million FB}
   $

Since the calculation closely matches option **A**, we should round as necessary based on the options provided.

Therefore, the correct option is 【A】.

To determine the impact on Confabulated's reported earnings before taxes in 2018, we need to analyze how the classification of Dumas SA affects financial reporting.

### Step-by-Step Analysis

1. **Classification Impact:**
   - Dumas SA is classified as "Amortized cost."

2. **Amortized Cost Accounting:**
   - Under this classification, the investment is held at its amortized cost on the balance sheet.
   - Changes in market value do not impact the income statement unless there is an impairment or the asset is sold.

3. **Market Value Change:**
   - The market value of Dumas SA increased from €54,000 in 2017 to €55,000 in 2018.
   - Since Dumas SA is held at amortized cost, this change in market value does not affect earnings.

4. **Earnings Impact:**
   - Since there are no sales or impairments noted, the change in market value does not affect Confabulated’s reported earnings before taxes for 2018.

Therefore, the correct option is 【A】.

...

To determine which option is NOT a possible reason for the violation of Covered Interest Parity (CIP) since the Great Financial Crisis, let's consider the potential reasons for such violations:

1. **Regulatory Changes**: Post-crisis regulations, such as those affecting banks' balance sheets, could influence CIP by impacting the cost of arbitrage.

2. **Market Liquidity**: Reduced liquidity in financial markets can lead to CIP deviations since fewer arbitrage opportunities are exploited.

3. **Risk Aversion**: Increased risk aversion among investors can lead to a demand for safer assets, affecting interest rate differentials and currency forward rates.

4. **Transaction Costs**: High transaction costs can prevent arbitrageurs from taking advantage of CIP deviations.

Given these potential reasons, the option that is least likely to explain a systematic violation of CIP would be one not related to the financial or regulatory environment affecting arbitrage opportunities.

Since "error" is not a detailed explanation related to market or regulatory factors, it is the option that does NOT fit as a possible reason.

Therefore, the correct option is 【error】.

To solve this question, we need to evaluate each strategy based on the interest-sensitive gap and the expected changes in interest rates.

1. **Bank A: Interest-sensitive gap = $50 million (positive)**
   - A positive gap means the bank has more interest-sensitive assets than liabilities.
   - If interest rates increase, the bank's income should increase because the return on assets will rise more than the cost of liabilities.
   - **Strategy I**: Bank A should increase investing in interest-sensitive assets if rates increase. This is appropriate because it would capitalize on the increased income from higher rates.

2. **Bank B: Interest-sensitive gap = -$30 million (negative)**
   - A negative gap means the bank has more interest-sensitive liabilities than assets.
   - If interest rates decrease, the bank's expenses should decrease more than its income, benefiting from the rate drop.
   - **Strategy II**: Bank B should decrease its interest-sensitive liabilities if rates decrease. This is appropriate because it would reduce the impact of falling rates on liabilities.

3. **Bank C:**
   - The strategy involves purchasing more assets with longer maturity and decreasing liabilities with longer maturity if rates fall.
   - This strategy is typically used when expecting rates to fall, as longer-term assets would lock in higher rates.
   - **Strategy III**: Appropriate if rates fall from 4% to 2%.

4. **Bank D:**
   - Decreasing the duration of assets and increasing the duration of liabilities when rates rise is not a typical strategy.
   - If rates rise, banks usually want to increase the duration of assets to benefit from higher rates over a longer period and decrease the duration of liabilities to mitigate cost.
   - **Strategy IV**: Not appropriate as described.

Therefore, the appropriate strategies are I, II, and III.

The correct option is 【C】.

To calculate the one-day 99.0% confident liquidity-adjusted value at risk (LVaR), we will follow these steps:

1. **Determine the Portfolio's Value at Risk (VaR):**

   - Calculate the z-score for a 99% confidence level from the standard normal distribution, which is approximately 2.33.

   - Use the formula for VaR:
     $
     \text{VaR} = \text{Portfolio Value} \times \text{Volatility} \times z
     $
     $
     \text{VaR} = \$4,000,000 \times 0.0259 \times 2.33
     $
     $
     \text{VaR} = \$241,988
     $

2. **Calculate the Liquidity-Adjusted Value at Risk (LVaR):**

   - Calculate the liquidity spread: 
     $
     \text{Liquidity Spread} = \text{Offer Price} - \text{Bid Price} = \$41 - \$39 = \$2
     $

   - Calculate the liquidity adjustment:
     $
     \text{Liquidity Adjustment} = \text{Liquidity Spread} \times \text{Number of Shares}
     $
     $
     \text{Liquidity Adjustment} = \$2 \times 100,000 = \$200,000
     $

   - Calculate the LVaR:
     $
     \text{LVaR} = \text{VaR} + \text{Liquidity Adjustment}
     $
     $
     \text{LVaR} = \$241,988 + \$200,000 = \$441,988
     $

3. **Select the Nearest Option:**

   - The nearest option to \$441,988 is \$400,000.

Therefore, the correct option is 【D】.

To determine the most suitable bond maturity strategy for the Rosenfeld Investment Firm's goal of using the portfolio as a source of liquidity, let's analyze each strategy:

### Strategies Overview

1. **Barbell Strategy**:
   - Involves investing in short-term and long-term bonds.
   - Provides flexibility but is not primarily focused on liquidity.

2. **Front-end Load Maturity Policy**:
   - Concentrates investments in short-term bonds.
   - Offers high liquidity due to the nature of short-term bonds.

3. **Back-end Load Maturity Policy**:
   - Focuses on long-term bonds.
   - Typically provides less liquidity due to longer maturities.

4. **Ladder Policy**:
   - Distributes investments evenly across various maturities.
   - Ensures regular cash flows and maintains liquidity.

### Analysis

- **Barbell Strategy**: Not the best for consistent liquidity.
- **Front-end Load Maturity Policy**: High liquidity due to short-term bonds.
- **Back-end Load Maturity Policy**: Not ideal for liquidity.
- **Ladder Policy**: Offers balanced liquidity through staggered maturities.

### Conclusion

For a goal focused on using the portfolio as a source of liquidity, the **Ladder Policy** is the most suitable strategy as it ensures regular cash flows and maintains liquidity.

Therefore, the correct option is 【D】.

To determine which statement is NOT correct, we need to calculate the weighted average of interest expense, break-even cost rate, overall cost of capital, and the least return rate the bank should earn.

### Given Data:
- Total interest expense = $50 million
- Total operating expense = $10 million
- Shareholder's investment = $150 million
- Total funds raised = $800 million
- All earning assets = $500 million
- Required return rate of shareholder (before tax) = 10%
- Tax rate = 35%

### Calculations:

1. **Weighted Average of Interest Expense:**
   $
   \text{Weighted Average Interest Rate} = \frac{\text{Total Interest Expense}}{\text{Total Funds Raised}} = \frac{50}{800} = 6.25\%
   $
   This matches statement A.

2. **Break-even Cost Rate:**
   Total cost includes interest, operating expenses, and shareholder return.
   $
   \text{Total Cost} = \text{Interest Expense} + \text{Operating Expense} + \text{Shareholder Return}
   $
   Shareholder return (after tax) = $ 150 \times 10\% \times (1 - 0.35) = 9.75 $
   
   $
   \text{Total Cost} = 50 + 10 + 9.75 = 69.75 \text{ million}
   $
   Break-even cost rate = $\frac{\text{Total Cost}}{\text{All Earning Assets}} = \frac{69.75}{500} = 13.95\%$

3. **Weighted Average Overall Cost of Capital:**
   $
   \text{Cost of Equity} = \frac{150}{800} \times 10\% = 1.875\%
   $
   $
   \text{Weighted Average Cost of Capital} = 6.25\% + 1.875\% = 8.125\%
   $
   Statement C claims 15%, which is incorrect.

4. **Least Return Rate the Bank Should Earn:**
   This should cover total costs, including tax adjustments.
   After-tax cost = $ 69.75 \times \frac{1}{1 - 0.35} = 107.308 $
   $
   \text{Least Return Rate} = \frac{107.308}{500} = 21.462\%
   $

Based on the calculations, the incorrect statement is the one related to the weighted average overall cost of capital being 15%.

Therefore, the correct option is 【C】.

In [7]:
import pandas as pd
import json
from pathlib import Path

# Load the JSON data from ModelOutputPath
model_output_path = Path(ModelOutputPath)
data1 = json.loads(model_output_path.read_text())
df1 = pd.DataFrame(data1)
model_output_path = Path(ModelRAGOutputPath)
data2 = json.loads(model_output_path.read_text())
df2 = pd.DataFrame(data2)

Evaluation

In [8]:
def evaluate(eva_file_path):
    data=json.loads(Path(eva_file_path).read_text())
    a=0
    for question in data:
        if question["Model Answer"] == question["Answer"]:
            a+=1
        else:
            a+=0
    accuracy=a/len(data)
    return accuracy

In [9]:
data1 = evaluate(ModelOutputPath)
data2 = evaluate(ModelRAGOutputPath)
print(data1,data2)

0.7061281337047354 0.7771587743732591


Retrieval